Chat Bot


In [1]:
import json
import numpy as np
import random
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder

In [3]:
lemmatizer = WordNetLemmatizer() # running to run turn word into base form

with open('./data/intens_tamil.json') as file:
    data = json.load(file)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

['வணக்கம்']
['ஹலோ']
['எப்படி', 'இருக்கிறீர்கள்', '?']
['நல்ல', 'நாள்']
['ஹாய்']
['என்ன', 'விசயம்', '?']
['பிரியாவிடை']
['பை']
['பின்னர்', 'சந்திப்போம்']
['அந்தவகையில்']
['நல்ல', 'நாள்']
['பார்க்கலாம்']
['நன்றி']
['மிக்க', 'நன்றி']
['உதவியது']
['மிக்க', 'நன்றிகள்']
['எப்போது', 'விவசாயம்', 'செய்யவேண்டும்', '?']
['விவசாயம்', 'செய்ய', 'ஏற்ற', 'நேரம்', 'என்ன', '?']
['நான்', 'எப்போது', 'பயிரிட', 'வேண்டும்', '?']
['பயிரிடும்', 'காலம்', 'என்ன', '?']
['எந்த', 'உரம்', 'பயன்படுத்த', 'வேண்டும்', '?']
['உரங்கள்', 'என்ன', '?']
['எப்படி', 'உரம்', 'இட', 'வேண்டும்', '?']
['உரம்', 'போடுவதற்கு', 'ஏற்ற', 'நேரம்', 'என்ன', '?']
['பயிர்களை', 'எப்படி', 'காப்பாற்றுவது', '?']
['பயிர்களை', 'பாதுகாப்பது', 'எப்படி', '?']
['பயிர்களுக்கான', 'பாதுகாப்பு', 'என்ன', '?']
['எப்படி', 'பயிர்களை', 'நசைவிலிருந்து', 'காப்பாற்றலாம்', '?']
['இன்று', 'வானிலை', 'எப்படி', 'இருக்கும்', '?']
['மழை', 'வருமா', '?']
['எப்போது', 'மழை', 'பெய்யும்', '?']
['விவசாயத்திற்கு', 'ஏற்ற', 'காலநிலை', 'என்ன', '?']
['எப்படி', 'நனைவு', 'செய்ய', 'வேண்

In [6]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(set(words))
classes = sorted(set(classes))
type(classes)

list

In [21]:
print(f"Classes: {classes}")
print(f"Words: {words}")

Classes: ['உரங்கள்', 'கீழ்பறுப்பு', 'நன்றி', 'நீர் வழங்கல்', 'பயிர்களின் காப்பு', 'பிரியாவிடை', 'வணக்கம்', 'வானிலை', 'வேளாண்மை நேரம்']
Words: ['அந்தவகையில்', 'அறுப்பது', 'அளவு', 'இட', 'இன்று', 'இருக்கிறீர்கள்', 'இருக்கும்', 'உதவியது', 'உரங்கள்', 'உரம்', 'எந்த', 'என்ன', 'எப்படி', 'எப்போது', 'எவ்வளவு', 'ஏற்ற', 'காப்பாற்றலாம்', 'காப்பாற்றுவது', 'காலநிலை', 'காலம்', 'கீழ்பறுப்பு', 'சந்திப்போம்', 'செய்ய', 'செய்யவேண்டும்', 'தேவை', 'தொழில்நுட்பம்', 'நசைவிலிருந்து', 'நனைவு', 'நன்றி', 'நன்றிகள்', 'நல்ல', 'நல்லது', 'நான்', 'நாள்', 'நீர்', 'நேரம்', 'பயன்படுத்த', 'பயிரிட', 'பயிரிடும்', 'பயிர்களுக்கான', 'பயிர்களுக்கு', 'பயிர்களை', 'பாதுகாப்பது', 'பாதுகாப்பு', 'பார்க்கலாம்', 'பின்னர்', 'பிரியாவிடை', 'பெய்யும்', 'பை', 'போடுவதற்கு', 'மழை', 'மிக்க', 'வணக்கம்', 'வருமா', 'வானிலை', 'விசயம்', 'விவசாயத்திற்கு', 'விவசாயம்', 'வேண்டும்', 'ஹலோ', 'ஹாய்']


In [22]:
# Training data
training = []
output_empty = [0] * len(classes)

In [23]:
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

In [24]:
random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               7936      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 9)                 585       
                                                                 
Total params: 16,777
Trainable params: 16,777
Non-trainable params: 0
_________________________________________________________________


In [26]:
history = model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5')


Epoch 1/200
8/8 [==============================] - 0s 3ms/step - loss: 2.2264 - accuracy: 0.0750
Epoch 2/200
8/8 [==============================] - 0s 2ms/step - loss: 2.2280 - accuracy: 0.1000
Epoch 3/200
8/8 [==============================] - 0s 2ms/step - loss: 2.1706 - accuracy: 0.1500
Epoch 4/200
8/8 [==============================] - 0s 2ms/step - loss: 2.1162 - accuracy: 0.2250
Epoch 5/200
8/8 [==============================] - 0s 2ms/step - loss: 2.0840 - accuracy: 0.2250
Epoch 6/200
8/8 [==============================] - 0s 2ms/step - loss: 2.0877 - accuracy: 0.2750
Epoch 7/200
8/8 [==============================] - 0s 2ms/step - loss: 2.1407 - accuracy: 0.2000
Epoch 8/200
8/8 [==============================] - 0s 2ms/step - loss: 2.0180 - accuracy: 0.4000
Epoch 9/200
8/8 [==============================] - 0s 2ms/step - loss: 2.0826 - accuracy: 0.2750
Epoch 10/200
8/8 [==============================] - 0s 2ms/step - loss: 1.9656 - accuracy: 0.3750
Epoch 11/200
8/8 [===========

In [27]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    print(sentence_words)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print(f"found in bag: {w}")
    return(np.array(bag))

def predict_class(sentence, model):
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result


In [28]:
print("Bot is running!")

while True:
    message = input("")
    if message == 'quit':
        break
    ints = predict_class(message, model)
    res = get_response(ints, data)
    print(res)

Bot is running!
['வணக்கம்']
1/1 [==============================] - 0s 45ms/step
வணக்கங்கள்!
['நீர்', 'அளவு', 'என்ன', '?']
1/1 [==============================] - 0s 31ms/step
மண் விதிவிலக்குகளை கவனித்து அதன் அடிப்படையில் நீர் அளவை மாற்றவும்.
['பயிர்களை', 'பாதுகாப்பது', 'எப்படி', '?']
1/1 [==============================] - 0s 28ms/step
நீங்கள் இடைவெளி பராமரிப்பு, பசும்பாலின் அளவு குறைத்தல் மற்றும் சரியான பண்ணை பயிர் முறைகளைப் பயன்படுத்தலாம்.
['பயிர்களை', 'பாதுகாப்பது', 'எப்படி']
1/1 [==============================] - 0s 17ms/step
பயிர்களைப் பொறுத்து அதன் தரத்தைப் பராமரிக்க உரங்களைச் சேர்த்து, கீறலைத் தவிர்க்கவும்.
['எந்த', 'உரம்', 'பயன்படுத்த', 'வேண்டும்']
1/1 [==============================] - 0s 25ms/step
நீங்கள் உங்கள் நிலத்தின் மண்ணின் புழுதியை ஆய்வு செய்து அதற்கான உரங்களை பயன்படுத்தலாம்.


Tamil Chatbot


In [2]:
import json
import numpy as np
import random
import tensorflow as tf
import nltk
from sklearn.preprocessing import LabelEncoder

# Download necessary NLTK data
# nltk.download('punkt')

# Custom normalization function for Tamil text
def tamil_normalize(word):
    suffixes = ['ம்', 'கள்', 'கள்', 'யை', 'களில்', 'களுக்கு', 'களால்', 'களிடம்', 'களை']
    for suffix in suffixes:
        if word.endswith(suffix):
            return word[:-len(suffix)]
    return word

def preprocess_sentence(sentence):
    words = nltk.word_tokenize(sentence)
    normalized_words = [tamil_normalize(word) for word in words]
    return normalized_words

# Load intents data
with open('./intens_tamil.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Prepare training data
words = []
classes = []
documents = []
ignore_words = ['?', '!', '.', ',']

for intent in data['intents']:
    for pattern in intent['patterns']:
        word_list = preprocess_sentence(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = sorted(set(words))
classes = sorted(set(classes))

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training, dtype=object)

train_x = np.array(list(training[:, 0]))
train_y = np.array(list(training[:, 1]))

# Build and compile model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model_tamil.h5')

# Functions to handle predictions and responses
def bow(sentence, words):
    sentence_words = preprocess_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence, model):
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result

# Chatbot loop



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               7808      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 9)                 585       
                                                                 
Total params: 16,649
Trainable params: 16,649
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
8/8 [==============================] - 1

In [3]:
print("Bot is running!")

while True:
    message = input("")
    if message == 'quit':
        break
    ints = predict_class(message, model)
    res = get_response(ints, data)
    print(res)

Bot is running!
1/1 [==============================] - 0s 17ms/step
விவசாய நேரம் உங்கள் பயிர்களின் வகையைக் கொண்டே மாறுபடும். உங்களது பகுதியின் சுழற்சியை கவனியுங்கள்.
1/1 [==============================] - 0s 17ms/step
பயிர்களைப் பொறுத்து அதன் தரத்தைப் பராமரிக்க உரங்களைச் சேர்த்து, கீறலைத் தவிர்க்கவும்.
1/1 [==============================] - 0s 20ms/step
பயிர்களின் வளர்ச்சிக்கு ஏற்றதாக நீங்கள் வாரத்திற்கு இரண்டு முறை மிதமான அளவில் நீர் கொடுக்க வேண்டும்.
